In [2]:
# import the necessary laibrary
import astropy.io.fits
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
# import logistic regression
from sklearn.linear_model import LogisticRegression
# import KNN classifier
from sklearn.neighbors import KNeighborsClassifier
# import SVM classifier
from sklearn.svm import SVC
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "simple_white"
import  plotly.express as px

In [3]:
# load the data
data = pd.read_csv("C:\\Users\omerl\OneDrive\Desktop\iml_repo\IML.HUJI\datasets\\agoda_cancellation_train.csv",engine='python')
data_frame = pd.DataFrame(data)

In [4]:
# get dummies for the categorical charge options
data_frame['charge_option'] = data_frame['charge_option'].apply(lambda x: 1 if x == 'Pay Now' else 0)

In [5]:
# describe the data
data_frame.describe()
data_frame.columns

Index(['h_booking_id', 'booking_datetime', 'checkin_date', 'checkout_date',
       'hotel_id', 'hotel_country_code', 'hotel_live_date',
       'hotel_star_rating', 'accommadation_type_name', 'charge_option',
       'h_customer_id', 'customer_nationality', 'guest_is_not_the_customer',
       'guest_nationality_country_name', 'no_of_adults', 'no_of_children',
       'no_of_extra_bed', 'no_of_room', 'origin_country_code', 'language',
       'original_selling_amount', 'original_payment_method',
       'original_payment_type', 'original_payment_currency',
       'is_user_logged_in', 'cancellation_policy_code', 'is_first_booking',
       'request_nonesmoke', 'request_latecheckin', 'request_highfloor',
       'request_largebed', 'request_twinbeds', 'request_airport',
       'request_earlycheckin', 'cancellation_datetime', 'hotel_area_code',
       'hotel_brand_code', 'hotel_chain_code', 'hotel_city_code'],
      dtype='object')

In [6]:
# fill missing values at cancellation_datetime column with zero values
data_frame['cancellation_datetime'] = data_frame['cancellation_datetime'].fillna(0)
data_frame['cancellation_datetime'] = (
    pd.to_numeric(data_frame['cancellation_datetime'],errors='coerce').fillna(1))
data_frame=data_frame.rename(columns = {'cancellation_datetime':'is_cancelled'})


In [7]:
# drop columns that are not relevant to the prediction
data_frame = data_frame.drop(['h_customer_id', 'h_booking_id', 'hotel_area_code'], axis=1)
data_frame = data_frame.drop(['original_payment_method', 'language','original_payment_currency'], axis=1)

In [8]:
# fill missing values with 0 at the following columns
data_frame['request_nonesmoke'] = data_frame['request_nonesmoke'].fillna(0)
data_frame['request_earlycheckin'] = data_frame['request_earlycheckin'].fillna(0)
data_frame['request_latecheckin'] = data_frame['request_latecheckin'].fillna(0)
data_frame['request_twinbeds'] = data_frame['request_twinbeds'].fillna(0)
data_frame['request_largebed'] = data_frame['request_largebed'].fillna(0)
data_frame['request_airport'] = data_frame['request_airport'].fillna(0)
data_frame['request_highfloor'] = data_frame['request_highfloor'].fillna(0)
data_frame['no_of_extra_bed'] = data_frame['no_of_extra_bed'].apply(lambda x: 1 if x>=1 else 0)
data_frame = data_frame.rename(columns = {'no_of_extra_bed':'request_extra_bed'})
### merge all request_* columns into one column
data_frame['request_twinbeds'] = data_frame['request_largebed'] + data_frame['request_twinbeds'] + data_frame['request_airport'] + data_frame['request_highfloor'] + data_frame['request_earlycheckin'] + data_frame['request_latecheckin'] + data_frame['request_nonesmoke']
# rename the request_twinbeds column to number_of_requests
data_frame = data_frame.rename(columns = {'request_twinbeds':'number_of_requests'})
data_frame['number_of_requests'] = data_frame['number_of_requests'].apply(lambda x: 0 if x < 1 else 1)
data_frame.drop(['request_largebed', 'request_airport', 'request_highfloor', 'request_earlycheckin', 'request_latecheckin', 'request_nonesmoke'], axis=1, inplace=True)
# print the correlation between the features and the is_cancelled column

In [9]:
dummy_2 = pd.get_dummies(data_frame['cancellation_policy_code'],prefix="cancel_policy")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
for col in dummy_2:
    if(dummy_2[col].sum() < 200):
        dummy_2.drop(col, inplace=True, axis=1)
for col in dummy_2:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.02):
        dummy_2.drop(col, inplace=True, axis=1)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)
# concat the dummies to the data_frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)

In [10]:

# # fill missing values of the hotel_chain_code column with "Unknown"
# data_frame['hotel_chain_code'] = data_frame['hotel_chain_code'].fillna("Unknown")
# # fill missing values of the hotel_brand_code column with "Unknown"
# data_frame['hotel_brand_code'] = data_frame['hotel_brand_code'].fillna("Unknown")
# dummy = pd.get_dummies(data_frame['hotel_chain_code'], prefix='hbc_')
# for col in dummy:
#     if(dummy[col].sum() < 200):
#         dummy.drop(col, inplace=True, axis=1)
# # conect the dummies to the data frame
# data_frame = pd.concat([data_frame, dummy], axis=1)
# # print the correlation between the features and the is_cancelled column
# data_frame.corr()['is_cancelled'].sort_values(ascending=False)

In [11]:
# explore the data in the Booking_datetime column
# and the checkin_date and checkout_date columns
# calculate the difference in days between the Booking_datetime and the checkin_date
# and the difference in days between the Booking_datetime and the checkout_date

data_frame['checkin_date'] = data_frame['checkin_date'].apply(pd.to_datetime)
data_frame['checkout_date'] = data_frame['checkout_date'].apply(pd.to_datetime)
data_frame['booking_datetime'] = data_frame['booking_datetime'].apply(pd.to_datetime)
# insert new column with the difference in days between the Booking_datetime and the checkin_date
data_frame['Lead_day'] = (data_frame['checkin_date'] - data_frame['booking_datetime']).dt.days
data_frame['stay_len'] = (data_frame['checkout_date'] - data_frame['checkin_date']).dt.days
# replace all the negative values with 0
data_frame['Lead_day'] = data_frame['Lead_day'].apply(lambda x: 0 if x < 0 else x)
data_frame['stay_len'] = data_frame['stay_len'].apply(lambda x: 0 if x < 0 else x)

In [12]:
data_frame['original_payment_type'] = data_frame['original_payment_type'].apply(lambda x: 0 if x == 'Credit Card' else 1)

In [13]:
dummy = pd.get_dummies(data_frame['origin_country_code'], prefix="origin_country_code")
dummy.insert(0, 'is_cancelled',data_frame['is_cancelled'])
dummy.fillna("none")
counter = 0
for col in dummy:
    if(dummy[col].sum() < 200):
        dummy.drop(col, inplace=True, axis=1)
for col in dummy:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy['is_cancelled'].corr(dummy[col])) < 0.01):
        dummy.drop(col, inplace=True, axis=1)
dummy.drop('is_cancelled', inplace=True, axis=1)
data_frame = pd.concat([data_frame, dummy], axis=1)

In [14]:
dummy_2 = pd.get_dummies(data_frame['guest_nationality_country_name'], prefix="guest_national")
dummy_2['is_cancelled'] = data_frame['is_cancelled']
for col in dummy_2:
    if(dummy_2[col].sum() < 100):
        dummy_2.drop(col, inplace=True, axis=1)
for col in dummy_2:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_2['is_cancelled'].corr(dummy_2[col])) < 0.01):
        dummy_2.drop(col, inplace=True, axis=1)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_2.drop('is_cancelled', inplace=True, axis=1)

In [15]:
# concat the dummies to the data frame
data_frame = pd.concat([data_frame, dummy_2], axis=1)
# data_frame.drop(['guest_nationality_country_name', 'customer_natio'], axis=1, inlpace=True)

In [16]:
print(data_frame.columns)
data_frame.drop(['cancellation_policy_code', 'checkin_date', 'booking_datetime','checkout_date', 'origin_country_code','guest_nationality_country_name','hotel_id','hotel_live_date', 'hotel_country_code','hotel_live_date','customer_nationality',
                 'guest_is_not_the_customer','guest_nationality_country_name',
                 'origin_country_code'], axis=1, inplace=True)

Index(['booking_datetime', 'checkin_date', 'checkout_date', 'hotel_id',
       'hotel_country_code', 'hotel_live_date', 'hotel_star_rating',
       'accommadation_type_name', 'charge_option', 'customer_nationality',
       'guest_is_not_the_customer', 'guest_nationality_country_name',
       'no_of_adults', 'no_of_children', 'request_extra_bed', 'no_of_room',
       'origin_country_code', 'original_selling_amount',
       'original_payment_type', 'is_user_logged_in',
       'cancellation_policy_code', 'is_first_booking', 'number_of_requests',
       'is_cancelled', 'hotel_brand_code', 'hotel_chain_code',
       'hotel_city_code', 'cancel_policy_0D0N', 'cancel_policy_14D100P',
       'cancel_policy_1D0N_100P', 'cancel_policy_1D100P',
       'cancel_policy_1D100P_100P', 'cancel_policy_1D1N_100P',
       'cancel_policy_1D1N_1N', 'cancel_policy_1D20P_100P',
       'cancel_policy_1D50P_100P', 'cancel_policy_2D100P',
       'cancel_policy_2D100P_100P', 'cancel_policy_2D1N_1N',
       'cancel

In [17]:
print(data_frame.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58659 entries, 0 to 58658
Data columns (total 79 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   hotel_star_rating                 58659 non-null  float64
 1   accommadation_type_name           58659 non-null  object 
 2   charge_option                     58659 non-null  int64  
 3   no_of_adults                      58659 non-null  int64  
 4   no_of_children                    58659 non-null  int64  
 5   request_extra_bed                 58659 non-null  int64  
 6   no_of_room                        58659 non-null  int64  
 7   original_selling_amount           58659 non-null  float64
 8   original_payment_type             58659 non-null  int64  
 9   is_user_logged_in                 58659 non-null  bool   
 10  is_first_booking                  58659 non-null  bool   
 11  number_of_requests                58659 non-null  int64  
 12  is_c

In [18]:
print(data_frame['accommadation_type_name'].value_counts())

Hotel                            42828
Resort                            5655
Guest House / Bed & Breakfast     2914
Hostel                            2633
Serviced Apartment                1470
Apartment                         1175
Motel                              573
Resort Villa                       357
Capsule Hotel                      320
Ryokan                             288
Bungalow                           113
Home                               106
UNKNOWN                             76
Private Villa                       62
Holiday Park / Caravan Park         42
Tent                                20
Love Hotel                          10
Boat / Cruise                        7
Lodge                                3
Inn                                  3
Homestay                             2
Chalet                               2
Name: accommadation_type_name, dtype: int64


In [26]:
dummy_3 = pd.get_dummies(data_frame['accommadation_type_name'], prefix="type_hotel")
dummy_3['is_cancelled'] = data_frame['is_cancelled']
print(dummy_3.columns)
for col in dummy_3:
    if(dummy_3[col].sum() < 100):
        dummy_3.drop(col, inplace=True, axis=1)
for col in dummy_3:
    if(col == 'is_cancelled'):
        continue
    if(np.abs(dummy_3['is_cancelled'].corr(dummy_3[col])) < 0.01):
        dummy_3.drop(col, inplace=True, axis=1)
#data_frame = pd.concat([data_frame, dummy], axis=1)
# print the correlation between the features and the is_cancelled column
dummy_3.drop('is_cancelled', inplace=True, axis=1)


Index(['type_hotel_Apartment', 'type_hotel_Boat / Cruise',
       'type_hotel_Bungalow', 'type_hotel_Capsule Hotel', 'type_hotel_Chalet',
       'type_hotel_Guest House / Bed & Breakfast',
       'type_hotel_Holiday Park / Caravan Park', 'type_hotel_Home',
       'type_hotel_Homestay', 'type_hotel_Hostel', 'type_hotel_Hotel',
       'type_hotel_Inn', 'type_hotel_Lodge', 'type_hotel_Love Hotel',
       'type_hotel_Motel', 'type_hotel_Private Villa', 'type_hotel_Resort',
       'type_hotel_Resort Villa', 'type_hotel_Ryokan',
       'type_hotel_Serviced Apartment', 'type_hotel_Tent',
       'type_hotel_UNKNOWN', 'is_cancelled'],
      dtype='object')


In [28]:
data_frame = pd.concat([data_frame, dummy_3], axis=1)
# print the correlation between the features and the is_cancelled column
print(data_frame.columns)
print(data_frame.info())
print(data_frame.isnull().sum())


Index(['hotel_star_rating', 'accommadation_type_name', 'charge_option',
       'no_of_adults', 'no_of_children', 'request_extra_bed', 'no_of_room',
       'original_selling_amount', 'original_payment_type', 'is_user_logged_in',
       'is_first_booking', 'number_of_requests', 'is_cancelled',
       'hotel_brand_code', 'hotel_chain_code', 'hotel_city_code',
       'cancel_policy_0D0N', 'cancel_policy_14D100P',
       'cancel_policy_1D0N_100P', 'cancel_policy_1D100P',
       'cancel_policy_1D100P_100P', 'cancel_policy_1D1N_100P',
       'cancel_policy_1D1N_1N', 'cancel_policy_1D20P_100P',
       'cancel_policy_1D50P_100P', 'cancel_policy_2D100P',
       'cancel_policy_2D100P_100P', 'cancel_policy_2D1N_1N',
       'cancel_policy_365D100P_100P', 'cancel_policy_3D100P',
       'cancel_policy_3D100P_100P', 'cancel_policy_3D1N_100P',
       'cancel_policy_3D50P_100P', 'cancel_policy_5D100P_100P',
       'cancel_policy_7D100P', 'cancel_policy_7D100P_100P', 'Lead_day',
       'stay_len', 'origi

In [29]:
data_frame.drop(['accommadation_type_name'], axis=1, inplace=True)

In [30]:
data_frame.to_csv('data_frame_final.csv')